In [1]:
try:
    import pyserial
except:
    import pip
    pip.main(['install', 'pyserial', 'fuzzywuzzy'])

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/home/bdtech/anaconda2/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
houseprices = pd.read_csv('houseprice.csv')
subway = pd.read_csv('subwayglasgow.csv')
crime = pd.read_csv('crimeglasgow.csv')

In [3]:
houseprices.head()

,CLASS,STNO,STnu,FLATPOSN,STNAME,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,QUARTER_(CALENDAR),ACTUAL PRICE AT POINT OF SALE,RPI,DEFLATOR,PRICE CONSTANT AT July 2013,ORIGINOFBUY,OMIT OR USE,NEWBUILD OR RESALE,LHF
0,R,55,NaN,NaN,"BRODIE DR,THE SIDINGS",G69 6ED,3.0,2012,2011/2012,01/03/2012,2012 Q1,159495.0,240.8,1.03696,165389.96,GLASGOW,Use,NEW,Baillieston Shettleston
1,R,99,NaN,NaN,"BRODIE DR,THE SIDINGS",G69 6FB,3.0,2012,2011/2012,01/03/2012,2012 Q1,155995.0,240.8,1.03696,161760.60,MOUNT VERNON,Use,NEW,Baillieston Shettleston
2,R,53,NaN,NaN,"BRODIE DR,THE SIDINGS",G69 6FB,3.0,2012,2011/2012,01/03/2012,2012 Q1,159495.0,240.8,1.03696,165389.96,GLASGOW,Use,NEW,Baillieston Shettleston
3,R,65,NaN,NaN,"BRODIE DR,THE SIDINGS",G69 6FD,3.0,2012,2011/2012,01/03/2012,2012 Q1,166495.0,240.8,1.03696,172648.68,GLASGOW,Use,NEW,Baillieston Shettleston
4,R,87,NaN,NaN,"BRODIE DR,THE SIDINGS",G69 6FD,3.0,2012,2011/2012,01/03/2012,2012 Q1,153495.0,240.8,1.03696,159168.20,BAILLIESTON,Use,NEW,Baillieston Shettleston


In [4]:
crime.head()

,Datazone,Intermediate geography name,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
0,S01003410,City Centre West,Anderston/City,Glasgow Kelvin,1,1,1,1,1,1
1,S01003373,City Centre West,Anderston/City,Glasgow Kelvin,2,2,4,2,2,2
2,S01003314,Parkhead West and Barrowfield,Calton,Glasgow Shettleston,15,6,11,4,3,3
3,S01003358,City Centre East,Anderston/City,Glasgow Kelvin,8,7,12,3,4,4
4,S01003433,City Centre West,Anderston/City,Glasgow Kelvin,39,30,16,6,6,5


In [5]:
subway.head()

,Station Name,Address Line 1,Address Line 2,Address Line 3,Postcode,Easting,Northing,Accessible,Park and Ride,Wi-Fi,Bike Shelter,Special Car Sharing Parking Spaces,Lost Property Number,Nearby
0,Bridge Street,67.0,Eglinton Street,Glasgow,G5 9NR,258710.0,664400.0,Yes,Yes,Yes,Yes,No,0141 335 3276.,"Citizen�s Theatre, 02 Academy."
1,Buchanan Street,174.0,Buchanan Street,Glasgow,G1 2JZ,259073.0,665577.0,Yes,No,Yes,No,No,0141 335 3276.,"Queen�s Street Station, Buchanan Bus Station, ..."
2,Cessnock,1.0,Cessnock Street,Glasgow,G51 1AS,256470.0,664490.0,Yes,No,Yes,No,No,0141 335 3276.,"Glasgow Science Centre, IMAX, SECC"
3,Cowcaddens,11.0,"Dundasvale Court, Unit 1",Glasgow,G4 0SN,258679.0,666191.0,Yes,No,Yes,No,No,0141 335 3276.,"Glasgow School of Art, GFT, Tenement House."
4,Govan,737.0,Govan Road,Glasgow,G51 2YJ,255490.0,665643.0,Yes,No,Yes,No,No,0141 335 3276.,"Southern General Hospital, Govan Old Parish Ch..."


In [6]:
#Cleaning the Data (Pre-processing)
houseprices.fillna('', inplace=True)
subway.fillna('', inplace=True)
crime.fillna('', inplace=True)

In [8]:
#Columns that are being renamed
houseprices.rename(columns={'LHF':'Place'}, inplace=True)
houseprices.rename(columns={'STNAME':'Street'}, inplace=True)
crime.rename(columns={'Intermediate geography name':'Place'}, inplace=True)
#Results shown below

In [9]:
houseprices.head()

,CLASS,STNO,STnu,FLATPOSN,Street,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,QUARTER_(CALENDAR),ACTUAL PRICE AT POINT OF SALE,RPI,DEFLATOR,PRICE CONSTANT AT July 2013,ORIGINOFBUY,OMIT OR USE,NEWBUILD OR RESALE,Place
0,R,55,,,"BRODIE DR,THE SIDINGS",G69 6ED,3,2012,2011/2012,01/03/2012,2012 Q1,159495,240.8,1.03696,165390,GLASGOW,Use,NEW,Baillieston Shettleston
1,R,99,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012,2011/2012,01/03/2012,2012 Q1,155995,240.8,1.03696,161761,MOUNT VERNON,Use,NEW,Baillieston Shettleston
2,R,53,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012,2011/2012,01/03/2012,2012 Q1,159495,240.8,1.03696,165390,GLASGOW,Use,NEW,Baillieston Shettleston
3,R,65,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012,2011/2012,01/03/2012,2012 Q1,166495,240.8,1.03696,172649,GLASGOW,Use,NEW,Baillieston Shettleston
4,R,87,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012,2011/2012,01/03/2012,2012 Q1,153495,240.8,1.03696,159168,BAILLIESTON,Use,NEW,Baillieston Shettleston


In [10]:
crime.head()

,Datazone,Place,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
0,S01003410,City Centre West,Anderston/City,Glasgow Kelvin,1,1,1,1,1,1
1,S01003373,City Centre West,Anderston/City,Glasgow Kelvin,2,2,4,2,2,2
2,S01003314,Parkhead West and Barrowfield,Calton,Glasgow Shettleston,15,6,11,4,3,3
3,S01003358,City Centre East,Anderston/City,Glasgow Kelvin,8,7,12,3,4,4
4,S01003433,City Centre West,Anderston/City,Glasgow Kelvin,39,30,16,6,6,5


In [12]:
#Indexing
crime['Place'] = crime['Place'].str.replace('Glasgow', '')
houseprices['Place'] = houseprices['Place'].str.replace('and', '')
subway['Postcode'] = subway['Postcode'].str.replace(r"[^a-zA-Z ]+", " ").str.strip()
subway['Postcode'] = subway['Postcode'].str.replace('\d+', '')
subway['Postcode'] = subway['Postcode'].str.replace('Glasgow', '')

In [13]:
subway.drop_duplicates(subset ="Postcode", 
                     keep = False, inplace = True) 

In [14]:
subway.head()

,Station Name,Address Line 1,Address Line 2,Address Line 3,Postcode,Easting,Northing,Accessible,Park and Ride,Wi-Fi,Bike Shelter,Special Car Sharing Parking Spaces,Lost Property Number,Nearby
0,Bridge Street,67,Eglinton Street,Glasgow,G NR,258710,664400,Yes,Yes,Yes,Yes,No,0141 335 3276.,"Citizen�s Theatre, 02 Academy."
1,Buchanan Street,174,Buchanan Street,Glasgow,G JZ,259073,665577,Yes,No,Yes,No,No,0141 335 3276.,"Queen�s Street Station, Buchanan Bus Station, ..."
2,Cessnock,1,Cessnock Street,Glasgow,G AS,256470,664490,Yes,No,Yes,No,No,0141 335 3276.,"Glasgow Science Centre, IMAX, SECC"
3,Cowcaddens,11,"Dundasvale Court, Unit 1",Glasgow,G SN,258679,666191,Yes,No,Yes,No,No,0141 335 3276.,"Glasgow School of Art, GFT, Tenement House."
4,Govan,737,Govan Road,Glasgow,G YJ,255490,665643,Yes,No,Yes,No,No,0141 335 3276.,"Southern General Hospital, Govan Old Parish Ch..."


In [16]:
houseprices.head()

,CLASS,STNO,STnu,FLATPOSN,Street,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,QUARTER_(CALENDAR),ACTUAL PRICE AT POINT OF SALE,RPI,DEFLATOR,PRICE CONSTANT AT July 2013,ORIGINOFBUY,OMIT OR USE,NEWBUILD OR RESALE,Place
0,R,55,,,"BRODIE DR,THE SIDINGS",G69 6ED,3,2012,2011/2012,01/03/2012,2012 Q1,159495,240.8,1.03696,165390,GLASGOW,Use,NEW,Baillieston Shettleston
1,R,99,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012,2011/2012,01/03/2012,2012 Q1,155995,240.8,1.03696,161761,MOUNT VERNON,Use,NEW,Baillieston Shettleston
2,R,53,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012,2011/2012,01/03/2012,2012 Q1,159495,240.8,1.03696,165390,GLASGOW,Use,NEW,Baillieston Shettleston
3,R,65,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012,2011/2012,01/03/2012,2012 Q1,166495,240.8,1.03696,172649,GLASGOW,Use,NEW,Baillieston Shettleston
4,R,87,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012,2011/2012,01/03/2012,2012 Q1,153495,240.8,1.03696,159168,BAILLIESTON,Use,NEW,Baillieston Shettleston


In [18]:
#Fuzz Ratio, below 
fuzz.ratio('Place', 'Postcode')

46

In [25]:
#Merge subway and houseprices columns
housing_and_subway = houseprices.merge(crime, 
                        left_on='Place', 
                        right_on='Place', 
                        how='outer', 
                        suffixes=["","_Place"] 
                       )
housing_and_subway.head(20)

,CLASS,STNO,STnu,FLATPOSN,Street,POSTCODE,MONTH OF SALE,YEAR OF SALE (CALENDAR),YEAR OF SALE (BUSINESS),MONTH AND YEAR,...,Place,Datazone,Multi-member ward name,2011 Scottish Parliamentary Constituencies,crime_deprivation_rank_2006,crime_deprivation_rank_2009,crime_deprivation_rank_2012,crime_local_rank_2006,crime_local_rank_2009,crime_local_rank_2012
0,R,55,,,"BRODIE DR,THE SIDINGS",G69 6ED,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R,99,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R,53,,,"BRODIE DR,THE SIDINGS",G69 6FB,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R,65,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R,87,,,"BRODIE DR,THE SIDINGS",G69 6FD,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R,95,,,"BROOMHOUSE CRES,KEEPERS G",G71 7RE,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,R,120,,,"BROOMHOUSE CRES,KEEPERS G",G71 7PH,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,R,31,,,"GREENOAKHILL CT,THE AVES",G71 7PQ,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,R,4,,,"GREENOAKHILL GATE,THE AVES",G71 7PR,3,2012.0,2011/2012,01/03/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,R,20,,,"BLACADER DR,ST CLEMENT",G69 8DG,1,2012.0,2011/2012,01/01/2012,...,Baillieston Shettleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Fuzz Ratio, below 
fuzz.ratio('Place', 'Street')

18